1. LIMPIEZA Y PREPARACIÓN DE LOS DATOS
2. CLUSTERIZACIÓN  

Para poder llevar a cabo el proyecto de machine learning lo primero que necesitábamos saber es si podíamos clasificar los países en segmentos diferentes. De esta manera, si podemos diferenciar a países por tener características diferentes, podríamos realizar regresiones para la eficiencia más fiables y a posteriori poder usar un modelo de clasificación para datos nuevos.
Para llevar a cabo esta fase hemos usado:  

***KMEANS***  
En primer lugar empezamos realizando la clusterización con todas las variables de que disponemos para generar un baseline con el que seguir avanzando y poder comparar ulteriores resultados. Dentro de los modelos vamos a usar varios tipos de escalados y en concreto, con la variable tipo de energía, vamos a probar con label encoder y con map de manera manual. Este punto está basado en que dados los diferentes niveles de contaminación que cada una de las fuentes de energía producen, podemos establecer una relación de orden entre ellas, de manera que aquellas que producen más co2 se catalogan con un valor superior a aquellas que producen menos.

PRIMERA CLUSTERIZACIÓN:  

*standard scaler, label encoder, todas las variables*:  

Encontramos que tenemos una división en 7 clusters, de los cuales 2 contienen la mayoría de los datos y 4 podrían considerarse residuales, los otros 2 restantes contienen datos en similar magnitud y se podrían considerar como clusters pequeños. De esta manera, consideramos que es viable el poder obtener una clusterización de 4 países de manera aceptable y que encaja con la idea general del inicio del proyecto, que aunque se basaba en 3 cluster, es posible que se nos esté diciendo que existe un segmento de países adicional al considerado.
Para poder corroborar la idea anterior llevamos a cabo la métrica denominada elbow, la cual en base a la inercia de los puntos respecto a los centroides, nos indicará cual puede ser el óptimo en nuestro dataset. Vemos que la variación de la inercia de un punto a otro comienza descender de manera menos pronunciada a partir del cluster 4, esto nos indica que el óptimo se encuentra en 4 cluster y por tanto podemos afirmar la consideración inicial como acertada dado que la suma de los cuadrados de la diferencia entre los puntos de cada cluster con su centroide (inercia) empieza a disminuir en menor proporción a partir del cluster 4. Lo que nos quiere decir esta menor disminución, es que si lo vemos desde el punto de vista de una tasa porcentual, es que la disminución porcentual que se produce a partir del cuarto cluster empieza a ser menor con cada cluster que se añade indicándonos que las diferencias en características cualitativas que tiene cada uno de los clusters generados empieza a difuminarse y por tanto a perder sentido la clusterización. Para poner en contexto esta idea, podemos llevarlo a l extremo de que siguieramos haciendo clusteres hasta que la inercia fuera 0. En este punto, lo que tendríamos sería un cluster por cada una de las entradas de nuestro dataset y por tanto, cada dato de manera individual sería un cluster. En este punto el objetivo de la clusterización se ha difuminado por completo y no tiene sentido, pues no hemos agrupado ningún dato en función de características similares, aunque nuestra inercia sea 0. Hemos perdido la información agregada de agrupar nuestros datos en segmentos diferenciados por características comunes. Además, en una segunda prueba elbow, en la que añadimos el tiempo de ejecución, vemos que con 4 clusters obtenemos el segundo mejor tiempo en coste computacional, lo que es un valor añadido a tener en cuenta, pues ahorra dinero.
Esta prueba inicial nos lleva a replantearnos la idea inicial que teníamos de división de los países en poco contaminantes, contaminantes y muy contaminantes a:

* poco contaminantes
* contaminación moderada
* contaminantes
* muy contaminantes

Realizamos la clusterización comentada y vemos que de nuevo obtenemos 2 clusters con la mayoría de los datos, uno pequeño y uno residual. Este residual puede ser debido a que, dado el conocimiento que tenemos de los datos por la realización del EDA, nuestro dataset contiene 3 países en concreto que tiene datos muy alejados del resto, como son China, Estados Unidos y Rusia, pues son los mayores productores mundiales de energía. Ellos solos proveen en conjunto la mayoría de la energía en el mundo. Por lo que seguiremos realizando pruebas a ver si este punto se confirma.  

SEGUNDA CLUSTERIZACIÓN:  

*standard scaler, 2 variables: emisiones de co2 y producción de energía:*  

Empezamos realizando la clusterización sin indicar número de clusters y con estas dos variables nos vuelve a realizar 7 cluster por defecto. Estas dos variables han sido elegidas en base a que son las más representativas del objeto de este proyecto y, por tanto, aquello que queremos representar con nuestros clusters. Sin embargo, ahora los datos están más repartidos entre los cluster, quedando como residuales solo dos que, aún siendo residuales, tienen más datos que los anteriores. Como hicimos en la anterior prueba, realizamos nuestra métrica del codo para revisar si estos clusters son óptimos y vemos que ahorael óptimo se sitúa en 3 clusters. Los realizamos y nos queda una división en la que prácticamente todos los datos se sitúan en el cluster 0, por lo que consideramos que no es una solución valida para nuestro modelo. Lo que ha pasado es que el cluster 0 y el 1, obtenido anteriormente con 4 clusters se han fusionado y por tanto, como estos clusters correspondían a los países poco contamianntes y con contaminación moderada, ahora han quedado todos como poco contaminantes.  

TERCERA CLUSTERIZACIÓN  

*standard scaler, 3 variables: eficiencia, producción de energía y tipo de energía:*  

Realizamos un nuevo modelo con 3 variables en las cuales incluimos la eficiencia de cada uno de los países, por incluir información sobre la emisión de co2, la producción de energía y el tipo de energía con el que se ha generado la misma. Elegimos estas variables, dado que son representativas de lo que se quiere representar con los clusters. En esta prueba, vemos que el óptimo mediante la métrica elbow vuelve a ser 4 clusters con una separación entre ellos parecida a la inicial con todas las variables, siendo la diferencia principal que con tres variables los cambios en la inercia son más acusados y por tanto es más fácil distinguir el óptimo. Esto nos da a entender que sí hay una diferencia importante entre los cluster y por tanto que el óptimo se pueda establecer en 4.
Por otra parte, representando los valores en un gráfico encontramos que la distribución de los mismos no tiene una forma circular. Por ello, como standard scaler nos hace un escalado en el que se centran los datos entorno a la media de la variable con desviación típica unitaria, estos datos escalados conservan la distribución original, que no es parecida a la normal, y es sensible a datos extemos. Así, lo que vamos a hacer es usar powertransformer con el transformador yeo-johnson, pues funciona con datos positivos y negativos, que nos escala los datos con media cero y desviación típica unitaria, haciendo que la distribución de los mismos se asemeje a una distribución Gaussiana, reduciendo la asimetría, que en nuestros datos es muy pronunciada al parecerse sus distribuciones, en su mayoría, a una distribución gamma. La consecuencia de todo esto es que la varianza de nuestras variables es muy elevada y powertransformer nos ayudará a estabilizarla.
Antes de realizar estos pasos para nuestro Kmeans, echamos un vistazo a la métrica silueta que es una métrica usada para medir la bondad del ajuste en clusterizaciones. Su valor oscila entre -1 y 1, siendo una métrica aceptable a partir de un score de 0.5. Obtener un 1 significaría que los clusters se diferencian bien unos de otros, 0 que la distancia entre los clusters no es significativa y por tanto es indiferente separarlos y -1 es que los clusters han sido asignados de manera incorrecta.
Sabiendo esto, hemos comprobado que el número óptimo de clusters está en 4 porque obtiene un score medio de silueta de 0.7, siendo de 0.3 en las demás clusterizaciones. Además, para 3 clusters no todos ellos están por encima de este score medio, mientras que en 4 sí lo están. Esto reafirma la elección de 4 clusters, pero aunque 4 clusters con standard scaler obtiene el mejor compromiso entre score general e individual de los clusters, queremos seguir probando con powertransformer y mapping para el tipo de energía.

CUARTA CLUSTERIZACIÓN  

*Power Transformer, todas las variables, label encoder:*  

Obtenemos 7 clusters como al inicio del estudio, pero repartidos de manera más homogénea. Además, mirando cual ha sido el cluster medio que se ha otorgado a cada país a lo largo de todos los años del dataset vemos que nos concuerda con las conclusiones extraídas del EDA previo. Así, mirando una muestra de estos países podemos ver lo siguiente:
Russia, Estados Unidos y China:
Han sido clasificados en media con un cuatro y se encuentran en la punta derecha de la gráfica del estudio donde están los países con mayores producciones y emisiones de co2, aunque no los que más.
Angola, África del sur:
son países con poca producción y pocas emisiones de co2, se han representado con un 1, y tiene sentido en relación a los datos del EDA.
Alemania, Reino Unido, Brasil:
Calificados con un 2 o 3 son países con producción media y emisiones de co2 media, tiene sentido.
Sin embargo, la representación en el gráfico no es homogénea y es difícil diferenciar los clusters. Esto nos está indicando que la definición de los mismos no es buena y por tanto resultará en un coeficiente de silueta pobre. Para ver si podemos mejorar esto, realizamos el siguiente estudio:

QUINTA CLUSTERIZACIÓN:

*Power Transformer,mapping,todas las variables*  

Usando igualmente todas las variables y asignando los pesos del encoding en la variable tipo de energía en función de la emisión de co2 media que tiene cada una de las fuentes de obtención de la energía vemos que:
China, Rusia y Estados Unidos:
se clasifica a China con casi un 4 de media que corresponde al carbón y es muy contaminante, a Rusia con casi un 2.78, que está a medio camino entre el gas y el petróleo, pero no llega al 3 porque produce más gas, y a Estados Unidos con un 3.6 que está entre el petróleo y el carbón. Esta clasificación se ajusta más a la realidad de los datos, si vemos que además sus medias no están muy alejadas las unas de las otras, como ocurre en su producción y emisión de co2 en los datos que tenemos situándolos como países con altas producciones y altas emisiones de co2, pero sin llegar a ser los más contaminantes en términos de eficiencia.
Nigeria, Colombia, Argentina:
se sitúan en torno al cluster 6 y 7 y en el anterior gráfico estaban en el cluster 2. Los 6 y 7 tienen producciones de energía en su mayoría pequeñas, aunque se encuentran a lo largo de todo el gráfico, y emisiones moderadas. En el aterior gráfico el 2 representaba producciones altas y emisiones altas por lo que en este sentido el mapping está haciendo mejor trabajo en la definición de los países de cada cluster, aunque 7 es demasiado elevado para los objetivos buscados con la clusterización.
Mirando los clusters óptimos mediante la métrica elbow vemos que sigue siendo 4 aunque el coste computacional ahora es el más elevado de los clusters.

SEXTA CLUSTERIZACIÓN  
*2 variables: emisión de co2 y producción de energía,power transoformer,mapping y label econder*  

Tanto con mapping como con label encoder la reducción de la complejidad del modelo a solo dos variables delimita claramente los clusters y obtenemos un gráfico muy interpretable. Sin embargo, con map es más clara que con label encoder pues podemos diferenciar mejor los clusters en base a las dos características elegidas y se ajusta mejor a la realidad obtenida en el EDA, donde hay más concentración de países en los clusters de emisiones bajas y menos concentración en los clusters de emisiones co2 y producción altas.

En conclusión dada la mayor interpretabilidad del modelo con 2 variables, la delimitación clara que se produce con kmeans y el mapping para la variable tipo de energía y el buen coeficiente de silueta obtenido en este modelo resulta ser el elegido.
Hemos podido corroborar que Kmeans trabaja bien con valores más dispersos, que es lo que realiza powertransformer cuando los asemejamos a la distribución normal y que aunque obtiene un score medio de silueta algo inferior a DBSCAN sus conclusiones son más robustas dado que todos los valores de silueta de sus cluster están por encima del score medio y por encima de 0.5 lo que nos indica que los segmentos de la población se diferencian suficientemente bien y son coherentes con las conlusiones sacadas del EDA. Además, la silueta o cluster más importante tiene un score cercano a 0.9 como pasa en DBSCAN reflejando el hecho de que existe un cluster mayoritario en el dataset.

***DBSCAN***  
En las representaciones de los datos usando las emisiones de co2 y la producción de energía, hemos podido apreciar que existe una gran desnsidad de datos y por tanto creemos que DBSCAN como algoritmo de clusterización puede hacer un buen trabajo dado que se basa en esta densidad de datos para separar los clusters. Se encarga de separalos identificando regiones menos densas estableciendo un radio alrededor de cada cluster y un mínimo de puntos que debe contener estas regiones densas. Además es un algoritmo que trabaja muy bien con valores extremos.  

*standard scaler, todas las variables:*  

Estalecemos el baseline con todas las variables y usando standard scaler. Con ello obtenemos 67 clusters, 473 puntos de ruido y un coeficiente de silueta de -0.001 lo que nos indica que esta clasificación no es válida. Para mejorar esto, lo que vamos ha hacer es intentar buscar un valor adecuado para el parámetro eps, que es el radio establecido en el cual se van a buscar las regiones densas de puntos. Es decir, los datos deben estar dentro de ese radio para ser considerados un cluster. Este dato es el más importante en DBSCAN. Mediante ello hemos encontrado que el óptimo lo encontraríamos en 4 clusters con un eps de 3.18 aproximadamente y un ruido de 47 datos. Para llegar a este punto, hemos buscado además cuál sería el valor óptimo de puntos mínimos a incluir en cada cluster mediante iteración y hemos obtenido un valor de 9 lo que nos ha dado como resultado un valor de silueta de 0.6  

*usando dos variables: emisiones de co2 y producción de energía, standard scaler:*  

Si usamos solamente las dos variables más representativas del proyecto, llegamos a conseguir un coeficiente de silueta medio de 0.85, por lo que nuestros clusters quedan muy bien definidos, usando 4 clusters. Sin embargo, el cluster 0 aglutina a prácticamente todos los datos del dataset y dado su elevado coeficiente, en la media este pesa mucho y consigue ese dato tan bueno. Lo que está pasando es que uno de los clusters tiene muchos datos pero el resto muy pocos y ese cluster tiene tanto peso que nos da una imagen general que parece muy buena, pero no nos convence, pues el resto tiene un score muy malo y no es representativo de nuestro dataset esta division de grupos.  

*2 variables, power transformer mapping y label encoder*

aunque obtenemos mejor coeficiente de silueta medio que con Kmeans para cuatro clusters, la dispersión que existe entre ellos es muy elevada, puesto que está apoyada en el gran peso que tiene uno solo de los cluster, el cual queda por encima del score medio de silueta, pero el resto de ellos está muy por debajo.

En conclusión podemos establecer que DBSCAN trabaja mejor con StandarScaler y label encoder porque las datos están más agrupados y por tanto tiene una distribución más densa, y es justamente donde este algoritmo lo hace mejor. Pero los resultados que obtenemos no se corresponden enteramente con las conclusiones extraidas del EDA. Además, aunque consigue un mayor valor medio de silueta, es debido al valor tan alto que alcanza uno solo del cluster dejando el resto por debajo del valor medio y por tanto nos dice que identifica muy bien un solo cluster, pero el resto los diferencia muy mal.
3. REGRESIÓN
4. CLASIFICACIÓN
5. MODELO FINAL